# Assignment 5
Simulate 1000 battle rounds in Risk.

Undertaken for the [Programming for Data Analytics](https://www.atu.ie/courses/higher-diploma-in-science-data-analytics#:~:text=Programming%20for%20Data%20Analytics) module as part of the [Higher Diploma in Science in Data Analytics](https://www.atu.ie/courses/higher-diploma-in-science-data-analytics) at ATU.

In [16]:
import numpy as np # to utilise random number generation
import matplotlib.pyplot as plt # for plotting

In [ ]:
# simulate a single round of Risk
def risk_round():
    # attacker and defender choose number of troops to put forward
    attack_dice = np.random.randint(1, 3)
    defend_dice = np.random.randint(1, 2)

  


In [18]:
risk_round()

Number of dice: 2
Dice rolls: [2, 4]
